In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [3]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [4]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [5]:
G = nx.Graph()
df = pd.read_excel('Agriculture/Agriculture_KAG_2016_17.xlsx')
adjacency_file = 'Karnataka_District_Adjacency_File.xlsx'

# existing_data = pd.DataFrame(df['District'])

existing_data = pd.read_csv('result.csv')

In [6]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [7]:
def get_node_stress(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
def get_node_stability(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [9]:
columns = ["Rice_Production_223"]
dim = len(columns)
init_graph_attr(G, "Karnataka_District_Adjacency_File.xlsx", df, columns)

# initialstress = get_node_stress(G,dim)
# df["Initial Stress"] = df["District"].map(initialstress)

In [10]:
# df["Initial Stress"]

In [11]:
def calculate_impact_score(df, base_column, capability_vector, intervention):
    # Perform simple linear regression
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()

    # Get coefficients and intercept
    m, c = model.params[base_column], model.params['const']
    print(c)
    # Predicted change in capability_vector for base_column + change_percentage% and base_column - change_percentage%
    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    rp_new_plus = m * base_column_increase + c 
    rp_new_minus = m * base_column_decrease + c 

    vector_plus = rp_new_plus - m * df[base_column] - c  
    vector_minus = rp_new_minus - m * df[base_column] - c

    # Generate normalized values using MinMaxScaler
    scaler = MinMaxScaler()

    normalized_vector_plus = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
    normalized_vector_minus = scaler.fit_transform(vector_minus.values.reshape(-1, 1))

    # Create new DataFrame with appropriate column names
    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{intervention}%)': rp_new_plus,
        f'{capability_vector} ({base_column} -{intervention}%)': rp_new_minus,
        f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_vector_plus.flatten(),
        f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_vector_minus.flatten()
    })

    return result_df

In [12]:
base_Column = "TotalNPK_315"
CapabilityVector = "Jowar_Yield_278"
change_percentage = 20

result_df = calculate_impact_score(df, base_Column, CapabilityVector, change_percentage)

result_df.head()

364.522399314813


,Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%)
0,639.044946,547.537431,0.098721,0.901279
1,591.336202,515.731601,0.075418,0.924582
2,452.520360,423.187707,0.007616,0.992384
3,787.129310,646.260340,0.171050,0.828950
4,1538.032682,1146.862588,0.537816,0.462184


In [13]:
def merge_columns_into_dataframe(result_df):
    for column in result_df.columns:
        if column not in existing_data.columns:
            existing_data[column] = result_df[column]
    return existing_data

# merge_columns_into_dataframe(result_df)
# existing_data.head()

In [14]:
result_df.head()

,Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%)
0,639.044946,547.537431,0.098721,0.901279
1,591.336202,515.731601,0.075418,0.924582
2,452.520360,423.187707,0.007616,0.992384
3,787.129310,646.260340,0.171050,0.828950
4,1538.032682,1146.862588,0.537816,0.462184


In [15]:
def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, change_percentage, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stability = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if change_percentage >= 0 else '-'} {abs(change_percentage)}%)"
    stability_column = existing_data["District"].map(NPK_stability)

    return stability_column, stability_column_name

# Example Usage:
columns_to_pass_minus20 = ["Normalized Rice_Production_223 (TotalNPK_315 -20%)"]
columns_to_pass_plus20 = ["Normalized Rice_Production_223 (TotalNPK_315 +20%)"]

# # Calculate and get stress column for NPK -20%
# stability_column_minus20, stability_column_name_minus20 = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_minus20, -20, dim)

# # Calculate and get stress column for NPK +20%
# stability_column_plus20, stability_column_name_plus20 = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_plus20, 20, dim)

# # Add stress columns to the result DataFrame
# existing_data[stability_column_name_plus20] = stability_column_plus20

# existing_data[stability_column_name_minus20] = stability_column_minus20

# Display the resulting DataFrame
existing_data.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.827968,0.827968


In [20]:
from bokeh.models import Span, Label

def calculate_and_visualize_impact_stability(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
    # Calculate Impact Score
    X = sm.add_constant(df[base_column])
    y = df[capability_vector]
    model = sm.OLS(y, X).fit()
    
    m, c = model.params[base_column], model.params['const']

    base_column_increase = (1 + intervention/100) * df[base_column]
    base_column_decrease = (1 - intervention/100) * df[base_column]

    change_vector_increase = m * base_column_increase + c
    change_vector_decrease = m * base_column_decrease + c

    scaler = MinMaxScaler()
    normalized_change_increase_vector = scaler.fit_transform(change_vector_increase.values.reshape(-1, 1))
    normalized_change_decrease_vector = scaler.fit_transform(change_vector_decrease.values.reshape(-1, 1))

    result_df = pd.DataFrame({
        f'{capability_vector} ({base_column} +{change_percentage}%)': change_vector_increase,
        f'{capability_vector} ({base_column} -{change_percentage}%)': change_vector_decrease,
        f'Normalized {capability_vector} ({base_column} +{change_percentage}%)': normalized_change_increase_vector.flatten(),
        f'Normalized {capability_vector} ({base_column} -{change_percentage}%)': normalized_change_decrease_vector.flatten()
    })

    # Merge Columns
    for column in result_df.columns:
        if column not in existing_data.columns:
            existing_data[column] = result_df[column]

    # Calculate Stability and add to DataFrame
    columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if change_percentage >= 0 else '-'}{abs(change_percentage)}%)"]
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, change_percentage, dim)

    existing_data[stability_column_name] = stability_column

    # Calculate Stability for the opposite change_percentage and add to DataFrame
    opposite_change_percentage = -change_percentage
    columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_change_percentage >= 0 else '-'}{abs(opposite_change_percentage)}%)"]
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_change_percentage, dim)

    existing_data[stability_column_name_opposite] = stability_column_opposite

    # Visualize with Bokeh
    source = ColumnDataSource(existing_data)
    plot = figure(title=f'Impact vs Stability ({capability_vector} - {base_column} {change_percentage}%)',
                  y_axis_label=f'Impact ({capability_vector})',
                  x_axis_label=f'Stability ({base_column} {change_percentage}%)',
                  tools='pan,box_zoom,wheel_zoom,reset,save')

    # Plot scatter points
    plot.scatter(y=f'Normalized {capability_vector} ({base_column} +{change_percentage}%)',
                x=stability_column_name,
                size=8,
                source=source)

    # Calculate average values from the ColumnDataSource
    avg_x = source.data[stability_column_name].mean()
    avg_y = source.data[f'Normalized {capability_vector} ({base_column} +{change_percentage}%)'].mean()

    # Create Span glyphs for average lines
    hline = Span(location=avg_y, dimension='width', line_color='red', line_width=2)
    vline = Span(location=avg_x, dimension='height', line_color='blue', line_width=2)

    # Add average lines to the plot
    plot.add_layout(hline)
    plot.add_layout(vline)

    # Add labels for average lines
    plot.add_layout(Label(x=avg_x, y=plot.y_range.end, text=f'Avg {stability_column_name}', text_color='red', text_font_size='10pt', text_align='center'))
    plot.add_layout(Label(x=plot.x_range.end, y=avg_y, text=f'Avg Normalized {capability_vector}', text_color='blue', text_font_size='10pt', text_align='center'))

    # Add tooltips
    hover = HoverTool()
    hover.tooltips = [("District", "@District")]
    plot.add_tools(hover)

    # Display the Bokeh plot in the Jupyter Notebook output cell
    show(plot, notebook_handle=True)
    return existing_data

base_Column = "TotalNPK_315"
CapabilityVector = "Rice_Production_223"
change_percentage = 20
dim = 1

existing_data = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
existing_data.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.827968,0.827968


In [21]:
# Save the resulting DataFrame to a new CSV file if needed
existing_data.to_csv('result.csv', index=False)

In [22]:
base_Column = "TotalNPK_315"
CapabilityVector = "Rice_Production_223"
change_percentage = 20
dim = 1

existing_data = calculate_and_visualize_impact_stability(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# Display the resulting DataFrame
existing_data.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%),New Stability(NPK - 20%)
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.959717,0.959717
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.973330,0.973330
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.853066,0.853066
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.733313,0.733313
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.827968,0.827968


In [23]:
# List of districts in the dataset
districts = df['District'].tolist()

# List of columns to visualize
columns_to_visualize = ['Rice_Production_223', 'Jowar_Yield_278', 'Maize_Production_226', 'SowingSeedsDistributed_Jowar_318', 'SowingSeedsDistributed_Maize_320']

colors = Category20[len(columns_to_visualize)]

# Create a figure for each column
for column, color in zip(columns_to_visualize, colors):
    # Create a new plot
    plot = figure(title=f'{column} vs District', x_range=districts, sizing_mode='stretch_width', toolbar_location=None, tools='')

    # Plot the data
    source = ColumnDataSource(data=dict(x=districts, y=df[column]))
    plot.vbar(x='x', top='y', width=0.8, color=color, legend_label=column, source=source)

    # Add hover tooltips
    hover = HoverTool()
    hover.tooltips = [('District', '@x'), (column, '@y')]
    plot.add_tools(hover)

    # Styling
    plot.xaxis.major_label_orientation = "vertical"
    plot.yaxis.axis_label = column
    plot.legend.location = 'top_right'
    plot.legend.click_policy = 'hide'

    # Show the plot
    show(plot)


In [24]:
from bokeh.models import FactorRange
from bokeh.transform import factor_cmap

# Assuming df is your DataFrame

# List of crops
crops = ['Rice_Production_223', 'Jowar_Yield_278', 'Maize_Production_226']

# Create a new plot
plot = figure(title='Production Comparison by Crop and District', x_range=FactorRange(*districts),toolbar_location=None, tools='')

# Plot the data
source = ColumnDataSource(df)
plot.vbar_stack(crops, x='District', width=0.9, color=Category20[len(crops)], source=source, legend_label=crops)

# Add hover tooltips
hover = HoverTool()
hover.tooltips = [('District', '@District')] + [(crop, f'@{crop}') for crop in crops]
plot.add_tools(hover)

# Styling
plot.xaxis.major_label_orientation = "vertical"
plot.yaxis.axis_label = "Production"
plot.legend.location = 'top_left'
plot.legend.click_policy = 'hide'

# Show the plot
show(plot)
